In [ ]:
import pandas as pd
import numpy as np
import os

directory = 'input/skypatrol2/'
files = sorted([f for f in os.listdir(directory) if f.endswith('.csv')])

results = []
for filename in files:
    filepath = os.path.join(directory, filename)
    df = pd.read_csv(filepath, comment='#')  # Skip comment lines starting with #
    
    # Sort by JD and calculate gaps
    jd_sorted = df['JD'].sort_values().values
    gaps = np.diff(jd_sorted)
    
    p99 = np.percentile(gaps, 99.73)
    results.append({
        'filename': filename.replace('-light-curves.csv', ''),
        'n_points': len(df),
        'p99_gap_days': p99
    })

results_df = pd.DataFrame(results)
results_df